In [2]:
from keras.applications import resnet50
import keras
import tensorflow as tf
import pandas as pd
import numpy as np
from skimage import io,transform
import os
import sys
model_name = 'ResNet_Incident'

Using TensorFlow backend.


#### Use the cell below if you need to regenerate the x, y (features, labels) arrays. Keep commented otherwise.

In [2]:
# counter = 0
# folder = '../Images/Incident/'
# data = pd.read_csv(os.path.join(folder, 'labels.csv'))
# imageFiles = os.listdir(folder)
# total = len(imageFiles)
# images = []
# labels = []
# size = 128
# for image_file in imageFiles:
#     if not image_file.endswith('.jpg'):
#         continue
#     img = io.imread(os.path.join(folder,image_file))
#     img = transform.resize(img, (size,size,3))
#     image_file = image_file.split('.')[0]
#     images.append(img)
#     labels.append(data[data.tid == float(image_file)].values[0][1:])
#     counter += 1
#     sys.stdout.write("\rProcessed: {0}/{1}".format(counter, total))
#     sys.stdout.flush()

# x = np.array(images)
# y = np.array(labels)


# np.save('../Images/Incident/incident_x%s.npy' % size, x)
# np.save('../Images/Incident/incident_y%s.npy' % size, y)

In [3]:
size = 128
x = np.load('../Images/Incident/incident_x128.npy')
y = np.load('../Images/Incident/incident_y128.npy')

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

In [5]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [15]:
from keras.layers import GlobalAveragePooling2D, Dropout, Dense
from keras.models import Model
base = resnet50.ResNet50(include_top=False, weights=None, classes=3, input_shape=(size,size,3), pooling='avg')
temp = base.output
temp = Dropout(0.5)(temp)
predictions = Dense(3, activation= 'sigmoid')(temp)
model = Model(inputs = base.input, outputs = predictions)

In [16]:
from keras.callbacks import ModelCheckpoint
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['acc'])

filepath="%s.best.hdf5" % model_name
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

hist = model.fit([x_train], y_train, batch_size=32, epochs=100, validation_data=([x_test], y_test), verbose=1, callbacks=callbacks_list)

Train on 630 samples, validate on 158 samples
Epoch 1/100
630/630 [==============================] - 6s 10ms/step - loss: 0.7915 - acc: 0.6561 - val_loss: 3.8834 - val_acc: 0.7468

Epoch 00001: val_loss improved from inf to 3.88339, saving model to ResNet_Incident.best.hdf5
Epoch 2/100
630/630 [==============================] - 2s 4ms/step - loss: 0.7441 - acc: 0.6820 - val_loss: 2.2646 - val_acc: 0.6646

Epoch 00002: val_loss improved from 3.88339 to 2.26462, saving model to ResNet_Incident.best.hdf5
Epoch 3/100
630/630 [==============================] - 2s 4ms/step - loss: 0.7642 - acc: 0.6688 - val_loss: 2.5091 - val_acc: 0.7511

Epoch 00003: val_loss did not improve from 2.26462
Epoch 4/100
630/630 [==============================] - 2s 4ms/step - loss: 0.7254 - acc: 0.6799 - val_loss: 2.5408 - val_acc: 0.7468

Epoch 00004: val_loss did not improve from 2.26462
Epoch 5/100
630/630 [==============================] - 2s 4ms/step - loss: 0.7610 - acc: 0.6508 - val_loss: 2.7723 - val_ac

630/630 [==============================] - 2s 4ms/step - loss: 0.0679 - acc: 0.9746 - val_loss: 0.9347 - val_acc: 0.7553

Epoch 00086: val_loss did not improve from 0.57005
Epoch 87/100
630/630 [==============================] - 2s 4ms/step - loss: 0.0485 - acc: 0.9852 - val_loss: 1.2161 - val_acc: 0.7637

Epoch 00087: val_loss did not improve from 0.57005
Epoch 88/100
630/630 [==============================] - 2s 4ms/step - loss: 0.0565 - acc: 0.9810 - val_loss: 0.9480 - val_acc: 0.7637

Epoch 00088: val_loss did not improve from 0.57005
Epoch 89/100
630/630 [==============================] - 2s 4ms/step - loss: 0.0517 - acc: 0.9847 - val_loss: 0.9187 - val_acc: 0.7595

Epoch 00089: val_loss did not improve from 0.57005
Epoch 90/100
630/630 [==============================] - 2s 4ms/step - loss: 0.0414 - acc: 0.9847 - val_loss: 0.9062 - val_acc: 0.7848

Epoch 00090: val_loss did not improve from 0.57005
Epoch 91/100
630/630 [==============================] - 2s 4ms/step - loss: 0.0441 

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("%s.json" % model_name, "w") as json_file:
    json_file.write(model_json)
    
# # serialize weights to HDF5
model.save_weights("%s.h5" % model_name)
print("Saved model to disk")

In [6]:
from keras.models import model_from_json
# load json and create model
json_file = open('%s.json' % model_name, 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("%s.h5" % model_name)
print("Loaded model from disk")

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['acc'])

Loaded model from disk


In [7]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, roc_auc_score
predicted = model.predict([x_test])
print(classification_report(y_test, predicted > 0.5))
print(precision_score(y_test, predicted > 0.5, average='micro'))
print(recall_score(y_test, predicted > 0.5, average='micro'))
print(roc_auc_score(y_test, predicted > 0.5, average='micro'))
print(f1_score(y_test, predicted > 0.5, average='micro'))

             precision    recall  f1-score   support

          0       0.43      0.14      0.21        22
          1       0.00      0.00      0.00        51
          2       0.88      0.32      0.47       111

avg / total       0.58      0.21      0.31       184

0.8125
0.21195652173913043
0.5904610194902549
0.3362068965517242


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
from keras.models import model_from_json
# load json and create model
json_file = open('%s.json' % model_name, 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("%s.best.hdf5" % model_name)
print("Loaded model from disk")

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['acc'])

Loaded model from disk


In [9]:
predicted = model.predict([x_test])
print(classification_report(y_test, predicted > 0.5))
print(precision_score(y_test, predicted > 0.5, average='micro'))
print(recall_score(y_test, predicted > 0.5, average='micro'))
print(roc_auc_score(y_test, predicted > 0.5, average='micro'))
print(f1_score(y_test, predicted > 0.5, average='micro'))

             precision    recall  f1-score   support

          0       0.33      0.32      0.33        22
          1       0.47      0.33      0.39        51
          2       0.83      0.80      0.82       111

avg / total       0.67      0.61      0.64       184

0.6890243902439024
0.6141304347826086
0.7191341829085456
0.6494252873563218
